In [7]:
import requests
import pandas as pd
from bcb import sgs, Expectativas
from natsort import natsort_key
import numpy as np


In [8]:
# função para fazer a formatação das DataFrames

def formata_df(df: pd.DataFrame):

    # Data to datetime
    df['Data'] = pd.to_datetime(df['Data'], format='%Y-%m-%d',)
    df['Data']= df['Data'].dt.strftime('%d/%m/%Y')
    
    # table pivot - table sorting by header - formating header
    df_pivot = df.pivot_table(index='Data', columns='Reuniao', values='Mediana')
    df_pivot = df_pivot.sort_index(axis=1)
    df_pivot = df_pivot.sort_index(key=lambda x: pd.to_datetime(x, format='%d/%m/%Y'))
    
    # Changing dtypes=float
    df_pivot = df_pivot.astype(float)    
    
    return df_pivot

In [9]:
# Endpoints
em = Expectativas()

ep_selic = em.get_endpoint('ExpectativasMercadoSelic')
ep_expec_anual = em.get_endpoint('ExpectativasMercadoAnuais')

In [10]:
# Selic mensal por reuniões

selic_mensal = ep_selic.query().filter(ep_selic.Data >= '2023-01-01', ep_selic.baseCalculo == '0').select(ep_selic.Data, ep_selic.Reuniao, ep_selic.Mediana).collect()
selic_mensal_pivot = formata_df(selic_mensal)

# natural_sort to sort header
idx = (selic_mensal_pivot.columns.to_series().str.split('/', expand=True)
         .sort_values(by=[1, 0], key=natsort_key).index
      )

selic_mensal_reuniao = selic_mensal_pivot[idx]

Reuniao,R1/2023,R2/2023,R3/2023,R4/2023,R5/2023,R6/2023,R7/2023,R8/2023,R1/2024,R2/2024,...,R1/2025,R2/2025,R3/2025,R4/2025,R5/2025,R6/2025,R7/2025,R8/2025,R1/2026,R2/2026
Data,,,,,,,,,,,,,,,,,,,,,
02/01/2023,13.75,13.75,13.75,13.75,13.75,13.25,12.75,12.25,11.75,11.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
03/01/2023,13.75,13.75,13.75,13.75,13.75,13.25,12.75,12.25,11.75,11.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
04/01/2023,13.75,13.75,13.75,13.75,13.75,13.25,12.75,12.25,11.75,11.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
05/01/2023,13.75,13.75,13.75,13.75,13.75,13.25,12.75,12.25,11.75,11.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
06/01/2023,13.75,13.75,13.75,13.75,13.75,13.25,12.75,12.25,11.75,11.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22/03/2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.0,9.0,8.75,8.50,8.5,8.5,8.5,8.5,8.5,8.5
25/03/2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.0,9.0,8.75,8.50,8.5,8.5,8.5,8.5,8.5,8.5
26/03/2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.0,9.0,8.75,8.50,8.5,8.5,8.5,8.5,8.5,8.5


In [13]:
# Selic anual
selic_anual = ep_expec_anual.query().filter(ep_expec_anual.Indicador == 'Selic', ep_expec_anual.Data >= '01/01/2023', ep_expec_anual.baseCalculo == '0').select(ep_expec_anual.Indicador, ep_expec_anual.Data, ep_expec_anual.DataReferencia, ep_expec_anual.Mediana).collect() 

# Data e DataRef to date time
selic_anual['DataReferencia'] = pd.to_datetime(selic_anual['DataReferencia'], format='%Y')
selic_anual['Data'] = pd.to_datetime(selic_anual['Data'], format='%Y-%m-%d')
selic_anual['Data'] = selic_anual['Data'].dt.strftime('%d/%m/%Y')

# table pivot - table sorting by header - formating header
selic_anual_pivot = selic_anual.pivot_table(index='Data', columns='DataReferencia', values='Mediana')    
selic_anual_pivot = selic_anual_pivot.sort_index(axis=1)
selic_anual_pivot.columns = selic_anual_pivot.columns.strftime('%Y')

# formating Index
selic_anual_pivot = selic_anual_pivot.sort_index(key=lambda x: pd.to_datetime(x, format='%d/%m/%Y'))

# Chaging dtypes=float
selic_anual_pivot = selic_anual_pivot.astype(float)



DataReferencia,2023,2024,2025,2026,2027,2028
Data,,,,,,
02/01/2023,12.25,9.00,8.0,8.0,8.0,NaN
03/01/2023,12.25,9.00,8.0,8.0,8.0,NaN
04/01/2023,12.25,9.00,8.0,8.0,8.0,NaN
05/01/2023,12.25,9.00,8.0,8.0,8.0,NaN
06/01/2023,12.25,9.25,8.0,8.0,8.0,NaN
...,...,...,...,...,...,...
22/03/2024,NaN,9.00,8.5,8.5,8.5,8.25
25/03/2024,NaN,9.00,8.5,8.5,8.5,8.25
26/03/2024,NaN,9.00,8.5,8.5,8.5,8.25


In [17]:
# to excel
with pd.ExcelWriter(r'C:\Users\rm170600\Documents\Python\Projeto_API_SGS\Expec_SELIC.xlsx') as writer:
   selic_mensal_reuniao.to_excel(writer, sheet_name='Mediana mensal')
   selic_anual_pivot.to_excel(writer, sheet_name='Mediana anual')